# 線性模型 (Linear Models) 完整指南

本 Notebook 將詳細介紹各種線性模型及其實務應用，包括：

1. **線性迴歸 (Linear Regression)**
   - 普通最小平方法 (OLS)
   - 梯度下降法 (以 `SGDRegressor` 示範)
2. **正則化線性迴歸 (Regularized Linear Regression)**
   - Ridge (L2 正則化)
   - Lasso (L1 正則化)
   - ElasticNet (L1 + L2 正則化)
3. **邏輯斯迴歸 (Logistic Regression)**
   - 二元分類
   - 多元分類
4. **廣義線性模型 (Generalized Linear Models)**
   - Poisson Regression (計數型資料)
   - Negative Binomial Regression (透過 statsmodels 展示)
5. **線性判別分析 (Linear Discriminant Analysis, LDA)**

此外，我們也將展示使用 pipeline 與 cross-validation 來評估模型效能。

最後，我們會簡要介紹其他非線性但常用於分類/回歸的模型（如 SVM、KNN）作為延伸，讓使用者了解當線性模型不適用時的替代方案。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid", font_scale=1.2)

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, ElasticNet,
                                 LogisticRegression, SGDRegressor)
from sklearn.metrics import (mean_squared_error, r2_score, accuracy_score,
                             classification_report, confusion_matrix)
from sklearn.datasets import load_diabetes, load_iris, make_regression, make_classification
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# statsmodels 用於 Poisson 與 Negative Binomial Regression
!pip install statsmodels > /dev/null
import statsmodels.api as sm

## 1. 線性迴歸 (Linear Regression)

線性迴歸假設特徵 $X$ 與目標 $y$ 之間存在線性關係，透過最小化 MSE 取得最佳係數。


### 1.1 普通最小平方法 (OLS)

In [ ]:
# 使用 diabetes 資料集作為回歸示範
diabetes = load_diabetes()
X = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y = diabetes.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_ols = LinearRegression()
model_ols.fit(X_train, y_train)
y_pred_ols = model_ols.predict(X_test)

mse_ols = mean_squared_error(y_test, y_pred_ols)
r2_ols = r2_score(y_test, y_pred_ols)

print("OLS 線性回歸結果：")
print(f"MSE: {mse_ols:.4f}")
print(f"R²: {r2_ols:.4f}")
print("Intercept:", model_ols.intercept_)
print("Coefficients:", model_ols.coef_)

In [ ]:
# 視覺化實際值 vs 預測值
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred_ols, alpha=0.5)
plt.plot([y_test.min(), y_test.max()],[y_test.min(), y_test.max()],'r--')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Linear Regression (OLS) Actual vs Predicted')
plt.show()

### 1.2 梯度下降法

使用 `SGDRegressor` 以梯度下降方式求解線性回歸。

In [ ]:
model_sgd = SGDRegressor(max_iter=1000, tol=1e-3, random_state=42)
model_sgd.fit(X_train, y_train)
y_pred_sgd = model_sgd.predict(X_test)

mse_sgd = mean_squared_error(y_test, y_pred_sgd)
r2_sgd = r2_score(y_test, y_pred_sgd)

print("SGD回歸結果：")
print(f"MSE: {mse_sgd:.4f}")
print(f"R²: {r2_sgd:.4f}")

## 2. 正則化線性迴歸

### Ridge, Lasso, ElasticNet

Ridge 加上 L2 正則化，Lasso 加上 L1 正則化，ElasticNet 結合 L1 與 L2 正則化。

In [ ]:
# Ridge
model_ridge = Ridge(alpha=1.0)
model_ridge.fit(X_train, y_train)
y_pred_ridge = model_ridge.predict(X_test)
print("Ridge R²:", r2_score(y_test, y_pred_ridge))

# Lasso
model_lasso = Lasso(alpha=1.0)
model_lasso.fit(X_train, y_train)
y_pred_lasso = model_lasso.predict(X_test)
print("Lasso R²:", r2_score(y_test, y_pred_lasso))

# ElasticNet
from sklearn.linear_model import ElasticNet
model_en = ElasticNet(alpha=1.0, l1_ratio=0.5)
model_en.fit(X_train, y_train)
y_pred_en = model_en.predict(X_test)
print("ElasticNet R²:", r2_score(y_test, y_pred_en))

可透過交叉驗證與網格搜尋來尋找最佳 alpha 參數。

---

## 3. 邏輯斯迴歸 (Logistic Regression)

邏輯斯迴歸用於分類問題（如二元分類），透過 sigmoid 函數將線性組合映射至 [0,1] 範圍，視為類別 1 的機率。

In [ ]:
# 使用 iris 資料集做邏輯斯迴歸示範(二元分類: 是否為virginica)
iris = load_iris()
X_clf = iris.data[:, 2:]  # 使用花瓣長度與花瓣寬度特徵
y_clf = (iris.target == 2).astype(int)  # 1代表virginica, 0代表其他

X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X_clf, y_clf, test_size=0.2, random_state=42)

model_log = LogisticRegression(max_iter=1000)
model_log.fit(X_train_clf, y_train_clf)
y_pred_log = model_log.predict(X_test_clf)

acc = accuracy_score(y_test_clf, y_pred_log)
print("Logistic Regression Accuracy:", acc)
print("\nClassification Report:\n", classification_report(y_test_clf, y_pred_log))

### 多元分類

邏輯斯迴歸也可用於多元分類任務（如原生的 iris 三類分類）。

In [ ]:
# 直接對全部iris資料進行多類別邏輯斯迴歸
X_mclf = iris.data
y_mclf = iris.target

X_train_mclf, X_test_mclf, y_train_mclf, y_test_mclf = train_test_split(X_mclf, y_mclf, test_size=0.2, random_state=42)

model_log_multi = LogisticRegression(max_iter=1000, multi_class='auto')
model_log_multi.fit(X_train_mclf, y_train_mclf)
y_pred_mclf = model_log_multi.predict(X_test_mclf)
acc_multi = accuracy_score(y_test_mclf, y_pred_mclf)
print("Multiclass Logistic Regression Accuracy:", acc_multi)
print("\nClassification Report:\n", classification_report(y_test_mclf, y_pred_mclf))

[可加入PCA降維可視化]

---

## 4. 廣義線性模型 (Generalized Linear Models)

### Poisson回歸
適用於計數資料的建模，目標變數為非負整數 (0,1,2,...)。


In [ ]:
# 使用 statsmodels 實現 Poisson Regression
# 生成 Poisson 資料
np.random.seed(42)
n_samples = 500
X_poi = np.random.normal(size=(n_samples, 2))
y_poi = np.random.poisson(np.exp(X_poi[:,0] + 0.5*X_poi[:,1]))

X_train_poi, X_test_poi, y_train_poi, y_test_poi = train_test_split(X_poi, y_poi, test_size=0.2, random_state=42)

# 使用 statsmodels GLM (廣義線性模型)
X_train_poi_sm = sm.add_constant(X_train_poi)
X_test_poi_sm = sm.add_constant(X_test_poi)

poisson_model = sm.GLM(y_train_poi, X_train_poi_sm, family=sm.families.Poisson())
poisson_res = poisson_model.fit()
print(poisson_res.summary())

y_pred_poi = poisson_res.predict(X_test_poi_sm)
mse_poi = mean_squared_error(y_test_poi, y_pred_poi)
print("Poisson回歸 MSE:", mse_poi)

### Negative Binomial Regression (負二項回歸)

在 Poisson 模型無法良好擬合過度離散 (Overdispersion) 的計數資料時，可考慮負二項回歸。

In [ ]:
# 模擬 Negative Binomial 資料 (此處僅簡單示意)
np.random.seed(42)
X_nb = np.random.normal(size=(n_samples, 2))
mu = np.exp(X_nb[:,0] + 0.3*X_nb[:,1])
alpha = 0.5 # 負二項分佈的額外參數
y_nb = np.random.negative_binomial(alpha, alpha/(alpha+mu))

X_train_nb, X_test_nb, y_train_nb, y_test_nb = train_test_split(X_nb, y_nb, test_size=0.2, random_state=42)

X_train_nb_sm = sm.add_constant(X_train_nb)
X_test_nb_sm = sm.add_constant(X_test_nb)

# statsmodels中可通過GLM配合NegativeBinomial family擬合
nb_model = sm.GLM(y_train_nb, X_train_nb_sm, family=sm.families.NegativeBinomial(alpha=alpha))
nb_res = nb_model.fit()
print(nb_res.summary())

y_pred_nb = nb_res.predict(X_test_nb_sm)
mse_nb = mean_squared_error(y_test_nb, y_pred_nb)
print("Negative Binomial回歸 MSE:", mse_nb)

---

## 5. 線性判別分析 (Linear Discriminant Analysis, LDA)

LDA 是一種線性分類技術，透過尋找將樣本投影到一條線上，使得投影後的類別間距離最大，類內距離最小。


In [ ]:
# 使用 iris 資料集做 LDA 示例
iris = load_iris()
X_lda = iris.data
y_lda = iris.target

X_train_lda, X_test_lda, y_train_lda, y_test_lda = train_test_split(X_lda, y_lda, test_size=0.2, random_state=42)

lda = LinearDiscriminantAnalysis()
lda.fit(X_train_lda, y_train_lda)
y_pred_lda = lda.predict(X_test_lda)
print("LDA Accuracy:", accuracy_score(y_test_lda, y_pred_lda))
print(classification_report(y_test_lda, y_pred_lda))

## 使用 Pipeline 與 Cross-Validation

以 Ridge 為例，透過 pipeline 與 cross_val_score 進行評估。

In [ ]:
X_rg, y_rg = make_regression(n_samples=500, n_features=10, noise=0.5, random_state=42)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('ridge', Ridge(alpha=1.0))
])

scores = cross_val_score(pipeline, X_rg, y_rg, cv=5, scoring='r2')
print("Cross-validation R² scores:", scores)
print("Mean R²:", np.mean(scores))

## 其他相關演算法簡介

雖然本 Notebook 聚焦於線性模型，但在實務中如果線性模型表現不佳，可以考慮其他演算法，例如：

- **支援向量機 (SVM)**：透過核函數在高維空間中尋找最大間隔的分隔超平面，用於分類與回歸。
- **K最近鄰 (KNN)**：基於距離度量找到測試樣本最近的 k 個鄰居，進行分類或回歸，適合較小維度與中小型資料集。
- **決策樹 (Decision Tree)** 與 **集成學習方法 (Random Forest, Gradient Boosting)**：適用於非線性、複雜的特徵空間。

若線性假設不成立，嘗試非線性模型常能帶來顯著的效能提升。

## 總結

本 Notebook 全面展示了線性模型在回歸與分類的應用，包括 OLS 線性回歸、梯度下降實現、Ridge/Lasso/ElasticNet 正則化迴歸、Logistic Regression、LDA，以及透過 GLM 實現 Poisson 和 Negative Binomial Regression。並提及其他非線性模型作為延伸。

實務中可根據資料特性、任務需求與模型表現來選擇適合的線性模型，並透過正則化、特徵工程、交叉驗證等方法優化模型表現。